In [2]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision torchaudio

     ---------------------------------------- 3.7/3.7 MB 9.1 MB/s eta 0:00:00
     ------------------------------------- 663.9/663.9 kB 10.5 MB/s eta 0:00:00
     -------------------------------------- 111.0/111.0 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 33.9 MB/s eta 0:00:00


In [333]:
import os
import h5py
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data, Batch
from torch_cluster import radius_graph
from rdkit import Chem
from rdkit.Chem import AllChem, rdchem, rdMolAlign
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.utils.data as torch_data    
import torch_geometric.data as pyg_data  
from torch_geometric.data import Batch

환경결정상수

In [334]:
# H5 파일 키 설정
RGD1_CONFIG = {
    "type": "rgd1",
    "keys": {
        "atoms": "elements", 
        "smiles": "Rsmiles",
        "R_coords": "RG", 
        "TS_coords": "TSG", 
        "PG_coords": "PG", # PG_coords 대신 PG 사용
        "R_energy": {"H":"R_H", "G":"R_F", "E":"R_E"}, # RE -> R_E, RGibbs -> R_F
        "TS_energy": {"H":"TS_H", "G":"TS_F", "E":"TS_E"},
        "P_energy": {"H":"P_H", "G":"P_F", "E":"P_E"}
    }
}

# 결합 해리 에너지 (BDE) 테이블 (kcal/mol)
BDE_TABLE = {
    (1, 1): 104, (1, 6): 99, (1, 7): 93, (1, 8): 111, (1, 9): 135,
    (6, 6): 83, (6, 7): 73, (6, 8): 86, (6, 9): 116,
    (7, 7): 39, (7, 8): 53, (8, 8): 35
}

# 원자 물성 [EN, Hardness, Radius, IE]
ATOM_PROPS_TABLE = {
    1: [2.20, 6.42, 0.31, 13.60], 6: [2.55, 5.00, 0.76, 11.26],
    7: [3.04, 7.27, 0.71, 14.53], 8: [3.44, 6.08, 0.66, 13.62], 9: [3.98, 7.01, 0.57, 17.42]
}
DEFAULT_PROPS = [0.0, 0.0, 0.0, 0.0]

# 작용기 SMARTS 패턴
FUNCTIONAL_GROUPS = {
    "Nitro": "[N+](=O)[O-]", "Sulfonyl": "S(=O)(=O)", "Cyano": "C#N", "Trifluoromethyl": "C(F)(F)F",
    "Carbonyl": "[CX3]=[OX1]", "Carboxyl": "[CX3](=O)[OX2H1]", "Halogen": "[F,Cl,Br,I]",
    "Amine": "[NX3;H2,H1,H0;!$(NC=O)]", "Hydroxy": "[OX2H]", "Alkoxy": "[OX2][CX4]", 
    "Amide": "[NX3][CX3](=O)", "Alkyl": "[CX4]", "Phenyl": "c1ccccc1"
}
FG_TYPE_MAP = {
    "Nitro": -1.0, "Sulfonyl": -1.0, "Cyano": -1.0, "Trifluoromethyl": -1.0,
    "Carbonyl": -0.5, "Carboxyl": -0.5, "Halogen": -0.2, "Amine": 1.0, "Hydroxy": 1.0, 
    "Alkoxy": 1.0, "Amide": 0.2, "Alkyl": 0.3, "Phenyl": 0.1
}

데이터셋

In [358]:
class ReactionDataset(torch_data.Dataset):
    def __init__(self, h5_path, config, limit=None):
        super().__init__()
        self.h5_path = h5_path
        self.config = config
        self.limit = limit
        self.data_keys = []
        if not os.path.exists(h5_path): raise FileNotFoundError(f"파일 없음: {h5_path}")
        
        with h5py.File(self.h5_path, 'r') as f:
            if self.config['type'] == 't1x':
                root = f['data']
                count = 0
                for formula in root.keys():
                    if self.limit and count >= self.limit: break
                    for rxn in root[formula].keys():
                        if self.limit and count >= self.limit: break
                        self.data_keys.append((formula, rxn))
                        count += 1
            else:
                self.data_keys = list(f.keys())[:self.limit]
        
        self.sym_map = {1:0, 6:1, 7:2, 8:3, 9:4, 'H':0, 'C':1, 'N':2, 'O':3, 'F':4,
                        b'H':0, b'C':1, b'N':2, b'O':3, b'F':4}

    def __len__(self): return len(self.data_keys)

    def get_atom_features(self, elements):
        """원소 리스트를 27차원 특성과 HSAB 특성으로 변환"""
        num_atoms = len(elements)
        x = torch.zeros((num_atoms, 27), dtype=torch.float)
        hsab = torch.zeros((num_atoms, 2), dtype=torch.float) # HSAB 기본값
        
        for i, el in enumerate(elements):
            val = el.decode() if isinstance(el, bytes) else el
            # Symbol One-hot
            idx = self.sym_map.get(val, 5)
            if idx < 6: x[i, idx] = 1.0
            x[i, 14] = 1.0 # 질량 가중치 등
            
            # HSAB 데이터 (전역 테이블 참조)
            # ATOM_PROPS_TABLE이 정의되어 있다면 값을 채움
            z_num = {0:1, 1:6, 2:7, 3:8, 4:9}.get(idx, 1)
            if 'ATOM_PROPS_TABLE' in globals():
                props = ATOM_PROPS_TABLE.get(z_num, [0.0, 0.0])
                hsab[i, 0] = -props[0]
                hsab[i, 1] = props[1]
        return x, hsab

    def __getitem__(self, idx):
        c = self.config
        key = self.data_keys[idx]
        try:
            with h5py.File(self.h5_path, 'r') as f:
                group = f[key] if c['type'] == 'rgd1' else f['data'][key[0]][key[1]]

                if c['type'] == 't1x':
                    z = torch.tensor(group['atomic_numbers'][()], dtype=torch.long)
                    x, hsab = self.get_atom_features(z.tolist())
                    traj = group['reactant']['positions']
                    pos_R = torch.tensor(traj[np.random.randint(0, len(traj))]).float()
                    pos_TS = torch.tensor(group['transition_state']['positions'][-1]).float()
                    pos_P = torch.tensor(group['product']['positions'][-1]).float()
                    e_key = c['energy_key']
                    e_R, e_TS, e_P = group['reactant'][e_key][-1], group['transition_state'][e_key][-1], group['product'][e_key][-1]
                    y_fwd = torch.tensor([e_TS - e_R]*3).float().unsqueeze(0)
                    y_rev = torch.tensor([e_TS - e_P]*3).float().unsqueeze(0)
                    thermo = torch.tensor([e_P - e_R]*3).float().unsqueeze(0)
                else:
                    kc = c['keys']
                    pos_R = torch.tensor(group[kc['R_coords']][()]).float()
                    pos_TS = torch.tensor(group[kc['TS_coords']][()]).float()
                    pos_P = torch.tensor(group[kc['PG_coords']][()]).float()
                    elems = group[kc['atoms']][()]
                    x, hsab = self.get_atom_features(elems)
                    z = torch.tensor([self.sym_map.get(e.decode() if isinstance(e, bytes) else e, 6) for e in elems], dtype=torch.long)
                    e_R, e_TS, e_P = float(group[kc['R_energy']['E']][()]), float(group[kc['TS_energy']['E']][()]), float(group[kc['P_energy']['E']][()])
                    y_fwd = torch.tensor([e_TS - e_R] * 3).float().unsqueeze(0)
                    y_rev = torch.tensor([e_TS - e_P] * 3).float().unsqueeze(0)
                    thermo = torch.tensor([e_P - e_R] * 3).float().unsqueeze(0)

                # 엣지 및 속성 완전 포함 (AttributeError 방지 핵심)
                edge_index = radius_graph(pos_R, r=5.0, max_num_neighbors=50)
                edge_bde = torch.zeros((edge_index.size(1), 1)).float()

                return pyg_data.Data(
                    x=x, z=z, hsab=hsab, 
                    pos=pos_R, pos_P=pos_P, y_pos=pos_TS, 
                    y_fwd=y_fwd, y_rev=y_rev, thermo_energy=thermo, 
                    edge_index=edge_index, edge_bde=edge_bde
                )
        except Exception as e:
            return None

GNN Backbone

In [359]:
class RBFLayer(nn.Module):
    """
    [Radial Basis Function Layer]
    거리 정보를 단순히 하나의 숫자가 아닌, 여러 개의 기저 함수(Gaussian)로 확장하여 표현합니다.
    이는 Geometric Deep Learning (SchNet, DimeNet)의 핵심 요소입니다.
    """
    def __init__(self, num_rbf=50, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        # 0 ~ cutoff 사이를 num_rbf 개수만큼 등분
        self.centers = nn.Parameter(torch.linspace(0, cutoff, num_rbf), requires_grad=False)
        self.widths = nn.Parameter(torch.tensor([cutoff / num_rbf] * num_rbf), requires_grad=False)

    def forward(self, dist):
        # Gaussian Smearing: exp(- (d - mu)^2 / beta)
        return torch.exp(-((dist - self.centers) ** 2) / self.widths)

In [360]:
class GeometricInteractionLayer(nn.Module):
    """
    [Geometric Interaction Block]
    RBF로 확장된 거리 정보를 이용하여 노드 간의 상호작용을 학습합니다.
    Simple MLP보다 훨씬 풍부한 기하학적 특징을 추출합니다.
    """
    def __init__(self, hidden_dim, num_rbf=50):
        super().__init__()
        self.rbf_lin = nn.Linear(num_rbf, hidden_dim)
        
        # Message Passing MLPs
        self.msg_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.update_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, h, edge_index, rbf_dist):
        row, col = edge_index
        
        # 1. 거리 정보 변환 (RBF -> Hidden)
        edge_geom = self.rbf_lin(rbf_dist) # [E, Hidden]
        
        # 2. 메시지 생성 (Source + Target) * Geometry
        # SchNet 방식의 Continuous Filter Convolution 아이디어 차용
        node_feat = torch.cat([h[row], h[col]], dim=-1)
        msg_raw = self.msg_net(node_feat)
        msg = msg_raw * edge_geom # Element-wise multiplication with Geometry
        
        # 3. 집계 (Aggregation)
        aggr = torch.zeros_like(h)
        aggr.index_add_(0, row, msg)
        
        # 4. 업데이트 (Update)
        update = self.update_net(torch.cat([h, aggr], dim=-1))
        
        return self.norm(h + update)

In [361]:
class GeometricGNN(nn.Module):
    """
    [Main Backbone]
    위상학적 정보뿐만 아니라 기하학적(RBF) 정보를 적극 활용하는 GNN입니다.
    """
    def __init__(self, node_dim, hidden_dim, num_layers=4):
        super().__init__()
        self.node_emb = nn.Linear(node_dim, hidden_dim)
        self.rbf = RBFLayer(num_rbf=50, cutoff=10.0)
        self.layers = nn.ModuleList([
            GeometricInteractionLayer(hidden_dim, num_rbf=50) for _ in range(num_layers)
        ])

    def forward(self, x, pos, edge_index):
        h = self.node_emb(x)
        
        # 거리 계산 및 RBF 변환
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        rbf_feat = self.rbf(dist) # [E, 50]
        
        for layer in self.layers:
            h = layer(h, edge_index, rbf_feat)
            
        return h

In [362]:
class SigmoidGating(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.gate_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 1), nn.Sigmoid()
        )
    def forward(self, h, edge_index, pos):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        return self.gate_mlp(torch.cat([h[row], h[col], dist], dim=-1))

In [363]:
class StructureDecoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.SiLU(), nn.Linear(hidden_dim, 1))
    def forward(self, h, edge_index, gate_score):
        row, col = edge_index
        return self.mlp(torch.cat([h[row], h[col]], dim=-1)) * gate_score

물리법칙

In [364]:
class PaulingBondLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.r0_table = torch.tensor([0.0, 0.31, 0.0, 0.0, 0.0, 0.0, 0.76, 0.71, 0.66, 0.57]) 
    def forward(self, z, edge_index, dist):
        row, col = edge_index
        r0_i = self.r0_table.to(z.device)[z[row]]
        r0_j = self.r0_table.to(z.device)[z[col]]
        r0_sum = (r0_i + r0_j).unsqueeze(-1)
        return torch.exp((r0_sum - dist) / 0.3)

In [365]:
class StereoelectronicLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos_R, pos_P, edge_index):
        row, col = edge_index
        vec_R = F.normalize(pos_R[col] - pos_R[row], p=2, dim=-1)
        vec_P = F.normalize(pos_P[col] - pos_P[row], p=2, dim=-1)
        return (vec_R * vec_P).sum(dim=-1, keepdim=True)

In [366]:
class HSABInteractionLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, hsab, edge_index):
        row, col = edge_index
        mu, eta = hsab[:, 0], hsab[:, 1]
        d_mu = (mu[row] - mu[col]).pow(2)
        sum_eta = eta[row] + eta[col]
        return (d_mu / (sum_eta + 1e-6)).unsqueeze(-1)

In [367]:
class StericLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos, edge_index):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1)
        density = 1.0 / (dist.pow(2) + 0.1)
        steric = torch.zeros(pos.size(0), 1, device=pos.device)
        steric.index_add_(0, row, density.unsqueeze(-1))
        return steric

In [368]:
class HammondGate(nn.Module):
    def __init__(self):
        super().__init__()
        self.gate = nn.Sequential(nn.Linear(3, 16), nn.SiLU(), nn.Linear(16, 1), nn.Sigmoid())
    def forward(self, thermo): return self.gate(thermo)

Diffusion

In [369]:
class SinusoidalTimeEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

In [370]:
class EGNNLayer(nn.Module):
    def __init__(self, hidden_dim, edge_attr_dim=0):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # [수정] edge_attr_dim 추가 (R/P와의 거리 정보 입력용)
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1 + edge_attr_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU()
        )
        
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.coord_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1, bias=False)
        )

    def forward(self, h, pos, edge_index, edge_attr=None):
        row, col = edge_index
        
        # (1) Radial Distance (Current State)
        coord_diff = pos[row] - pos[col]
        radial = torch.sum(coord_diff**2, dim=1, keepdim=True)
        
        # (2) Edge Feature Construction
        # [수정] edge_attr(조건 정보)가 있으면 함께 Concatenate
        edge_feats = [h[row], h[col], radial]
        if edge_attr is not None:
            edge_feats.append(edge_attr)
            
        edge_input = torch.cat(edge_feats, dim=1)
        msg = self.edge_mlp(edge_input)
        
        # (3) Coordinate Update (Equivariant)
        trans = coord_diff * self.coord_mlp(msg)
        agg_trans = torch.zeros_like(pos)
        agg_trans.index_add_(0, row, trans)
        pos_new = pos + agg_trans
        
        # (4) Node Feature Update
        agg_msg = torch.zeros_like(h)
        agg_msg.index_add_(0, row, msg)
        node_input = torch.cat([h, agg_msg], dim=1)
        h_new = h + self.node_mlp(node_input)
        
        return h_new, pos_new

In [371]:
class EquivariantDenoisingNetwork(nn.Module):
    def __init__(self, node_dim, hidden_dim, num_layers=4):
        super().__init__()
        
        # Time Embedding
        self.time_mlp = nn.Sequential(
            SinusoidalTimeEmbeddings(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.emb = nn.Linear(node_dim, hidden_dim)
        
        # [수정] EGNNLayer에 edge_attr_dim=2 추가 (dist_to_R, dist_to_P)
        self.layers = nn.ModuleList([
            EGNNLayer(hidden_dim, edge_attr_dim=2) for _ in range(num_layers)
        ])
        
        self.final_coord_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1, bias=False)
        )

    def forward(self, h_raw, pos_t, t, edge_index, pos_R, pos_P):
        # 1. Embeddings
        h = self.emb(h_raw)
        time_emb = self.time_mlp(t)
        
        # Time Injection (Simple addition for broadcasting)
        # Note: In production, gather time_emb by batch index
        # h = h + time_emb[batch_index] logic assumes implicit broadcasting or mapped externally
        
        # 2. Conditioning Features (Geometry-Awareness)
        # 현재 노이즈 상태(pos_t)가 R과 P로부터 얼마나 떨어져 있는지 계산
        row, col = edge_index
        
        # Edge: Distance to Reactant (relative difference projected on edge?)
        # 더 간단하고 강력한 방법: Node별 거리를 Edge로 확장
        # 여기서는 Edge Feature로 "두 노드 간 거리" 대신 "R/P 구조에서의 거리"를 참고하도록 함
        
        # (A) R/P 구조에서의 Pairwise Distance (Invariant Conditioning)
        dist_R = (pos_R[row] - pos_R[col]).norm(dim=-1, keepdim=True)
        dist_P = (pos_P[row] - pos_P[col]).norm(dim=-1, keepdim=True)
        
        # 이를 Edge Attribute로 사용
        cond_attr = torch.cat([dist_R, dist_P], dim=-1) # [E, 2]
        
        # 3. EGNN Layers
        curr_pos = pos_t
        curr_h = h
        
        for layer in self.layers:
            # Time embedding injection could be added inside layer, 
            # here we assume h already carries some state or we add it:
            # curr_h = curr_h + time_emb_mapped
            curr_h, curr_pos = layer(curr_h, curr_pos, edge_index, edge_attr=cond_attr)
            
        noise_pred = curr_pos - pos_t 
        return noise_pred

In [372]:
class TSDiffusionModule(nn.Module):
    def __init__(self, node_dim, hidden_dim, timesteps=100):
        super().__init__()
        self.timesteps = timesteps
        
        # Noise Schedule (Linear)
        beta_start, beta_end = 1e-4, 0.02
        betas = torch.linspace(beta_start, beta_end, timesteps)
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.register_buffer('betas', betas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1. - alphas_cumprod))
        
        # Denoising Network (기존 Conditioned 구조 유지)
        self.denoise_net = EquivariantDenoisingNetwork(node_dim, hidden_dim)

    def compute_loss(self, true_ts_pos, x_feat, edge_index, batch, pos_R, pos_P):
        batch_size = batch.max().item() + 1
        device = true_ts_pos.device
        
        # 1. Random Timestep & Noise
        t = torch.randint(0, self.timesteps, (batch_size,), device=device).long()
        noise = torch.randn_like(true_ts_pos)
        
        # 2. Forward Diffusion Process (add noise)
        t_node = t[batch]
        sqrt_alpha = self.sqrt_alphas_cumprod[t_node].unsqueeze(-1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t_node].unsqueeze(-1)
        
        noisy_pos = sqrt_alpha * true_ts_pos + sqrt_one_minus_alpha * noise
        
        # 3. Predict Noise (Conditioned on R and P)
        noise_pred = self.denoise_net(x_feat, noisy_pos, t, edge_index, pos_R, pos_P)
        
        # --- [Strategy 1] Active Atom Weighting (Object-Aware) ---
        # 반응물(R)과 생성물(P) 사이의 이동 거리를 계산
        displacement = (pos_R - pos_P).norm(dim=-1, keepdim=True) # [N, 1]
        
        # 이동 거리가 0.5 Angstrom 이상인 원자(반응 중심)에 가중치 3.0 부여
        # 그 외 원자(Spectator)는 가중치 1.0
        # 논문 'Accurate transition state generation...'의 핵심 아이디어 적용
        atom_weights = torch.where(displacement > 0.5, 3.0, 1.0)
        
        # Weighted MSE Loss (Coordinate Loss)
        loss_coord = (atom_weights * (noise_pred - noise) ** 2).mean()
        
        # --- [Strategy 2] Auxiliary Distance Loss (Geometry Consistency) ---
        # 예측된 노이즈로 "복원된 구조(Denoised x0)"를 추정
        # x0_pred = (xt - sqrt(1-a)*eps_pred) / sqrt(a)
        pred_x0 = (noisy_pos - sqrt_one_minus_alpha * noise_pred) / (sqrt_alpha + 1e-8)
        
        # 실제 TS와 예측된 TS의 "원자 간 거리(Pairwise Distance)" 비교
        # 논문 'Generating transition states... via distance-geometry' 아이디어 적용
        row, col = edge_index
        d_pred = (pred_x0[row] - pred_x0[col]).norm(dim=-1)
        d_true = (true_ts_pos[row] - true_ts_pos[col]).norm(dim=-1)
        
        # 거리 행렬 오차 추가 (구조적 강체성 유지에 도움)
        loss_dist = F.mse_loss(d_pred, d_true)
        
        # 최종 Loss: 좌표 정확도 + 기하학적 거리 일관성
        return loss_coord + 0.5 * loss_dist

    @torch.no_grad()
    def sample(self, pos_R, pos_P, x_feat, edge_index, batch):
        # Initialization: Interpolation + Noise (Better initialization than pure noise)
        # 전략 문서의 'Initialization' 부분 반영: R/P 보간 지점에서 시작
        current_pos = (pos_R + pos_P) / 2.0 + torch.randn_like(pos_R) * 0.1
        
        for t in reversed(range(self.timesteps)):
            batch_size = batch.max().item() + 1
            t_batch = torch.full((batch_size,), t, device=pos_R.device, dtype=torch.long)
            
            # Predict Noise
            noise_pred = self.denoise_net(x_feat, current_pos, t_batch, edge_index, pos_R, pos_P)
            
            # Update (DDPM)
            alpha = 1 - self.betas[t]
            alpha_cumprod = self.alphas_cumprod[t]
            sigma = torch.sqrt(self.betas[t])
            
            mean = (1 / torch.sqrt(alpha)) * (
                current_pos - (self.betas[t] / torch.sqrt(1 - alpha_cumprod)) * noise_pred
            )
            
            if t > 0:
                noise = torch.randn_like(current_pos)
                current_pos = mean + sigma * noise
            else:
                current_pos = mean
                
        return current_pos

Predictor

In [373]:
class PhysChemInformedPredictor(nn.Module):
    def __init__(self, hidden_dim, dropout=0.1):
        super().__init__()
        self.pauling = PaulingBondLayer()
        self.stereo = StereoelectronicLayer()
        self.hsab = HSABInteractionLayer()
        self.steric = StericLayer()
        self.hammond = HammondGate()
        
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim + 4, hidden_dim), nn.SiLU(), nn.Dropout(dropout), nn.Linear(hidden_dim, 3)
        )
        # Edge Feats: 2H + Dist(1)*2 + Gate(1) + Pauling(3) + Stereo(1) + HSAB(1) + BDE(1) + Hammond(1) + PredD(1) = 2H + 11
        edge_in = (hidden_dim * 2) + 11
        self.edge_mlp = nn.Sequential(
            nn.Linear(edge_in, hidden_dim), nn.SiLU(), nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 6) # Fwd(3) + Rev(3)
        )

    def forward(self, h, z, hsab, edge_index, pos_R, pos_P, gate_score, pred_delta_D, batch, thermo, bde):
        row, col = edge_index
        late_char = self.hammond(thermo); late_char_edge = late_char[batch[row]]
        
        steric_R = self.steric(pos_R, edge_index)
        steric_graph = global_add_pool(steric_R, batch)
        h_graph = global_add_pool(h, batch)
        base = self.node_mlp(torch.cat([h_graph, thermo, steric_graph], dim=-1))
        
        dist_R = (pos_R[row]-pos_R[col]).norm(dim=-1, keepdim=True)
        dist_P = (pos_P[row]-pos_P[col]).norm(dim=-1, keepdim=True)
        
        bo_R = self.pauling(z, edge_index, dist_R)
        bo_P = self.pauling(z, edge_index, dist_P)
        bo_TS = (1.0 - late_char_edge) * bo_R + late_char_edge * bo_P
        align = self.stereo(pos_R, pos_P, edge_index)
        hsab_E = self.hsab(hsab, edge_index)
        
        edge_feat = torch.cat([
            h[row], h[col], dist_R, dist_P, gate_score,
            bo_R, bo_P, bo_TS, bde, align, late_char_edge, hsab_E, pred_delta_D
        ], dim=-1)
        
        edge_out = self.edge_mlp(edge_feat) * gate_score
        barriers = global_add_pool(edge_out, batch[row])
        
        return barriers[:, :3] + base, barriers[:, 3:] + base, late_char

Model

In [374]:
class BEPGeoModel(nn.Module):
    def __init__(self, node_dim=27, hidden_dim=256, num_layers=4):
        super().__init__()
        self.node_emb = nn.Linear(node_dim, hidden_dim)
        # [교체] 기능이 강화된 Diffusion Module 사용
        self.diffusion = TSDiffusionModule(hidden_dim, hidden_dim)
        self.gating = SigmoidGating(hidden_dim)
        self.energy_predictor = PhysChemInformedPredictor(hidden_dim)

    def forward(self, data):
        h = self.node_emb(data.x)
        
        if self.training:
            # [수정] Diffusion 학습 시 pos_R, pos_P를 함께 전달하여 
            # Object-Aware Weighting 계산에 사용
            struct_loss = self.diffusion.compute_loss(
                data.y_pos, h, data.edge_index, data.batch, data.pos, data.pos_P
            )
            
            # Energy Predictor용 Noise Injection (Robustness)
            noise_level = 0.1
            ts_pos_input = data.y_pos + torch.randn_like(data.y_pos) * noise_level
            
            dist_R = (data.pos[data.edge_index[0]] - data.pos[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS = (ts_pos_input[data.edge_index[0]] - ts_pos_input[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            pred_delta_D = dist_TS - dist_R

        else:
            struct_loss = 0.0
            # Sampling 시에도 R/P Conditioning 활용
            generated_ts = self.diffusion.sample(
                data.pos, data.pos_P, h, data.edge_index, data.batch
            )
            ts_pos_input = generated_ts
            
            dist_R = (data.pos[data.edge_index[0]] - data.pos[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS_gen = (generated_ts[data.edge_index[0]] - generated_ts[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            pred_delta_D = dist_TS_gen - dist_R

        gate_map = self.gating(h, data.edge_index, data.pos)
        
        thermo_sq = data.thermo_energy.squeeze(1) 
        pred_fwd, pred_rev, ts_char = self.energy_predictor(
            h, data.z, data.hsab, data.edge_index, data.pos, data.pos_P,
            gate_map, pred_delta_D, data.batch, thermo_sq, data.edge_bde
        )
        
        return struct_loss, pred_fwd, pred_rev, gate_map, ts_char, pred_delta_D

Trainer

In [375]:
class BEPTrainer:
    def __init__(self, model, optimizer, device, pretrain_mode=False):
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.pretrain_mode = pretrain_mode

    def train_step(self, batch):
        self.model.train()
        batch = batch.to(self.device)
        self.optimizer.zero_grad()
        
        diff_loss, pred_fwd, pred_rev, gate_logits, _, _ = self.model(batch)
        
        if self.pretrain_mode:
            total_loss = diff_loss
            return total_loss.item(), torch.zeros(1), torch.zeros(1)
        
        # Fine-tuning: 구조 + 게이트 + 에너지
        target_fwd = batch.y_fwd * 627.5 # kcal/mol 변환
        l_energy = F.smooth_l1_loss(pred_fwd, target_fwd.squeeze(1))
        
        # Gating Loss (Logits 기반)
        dist_R = (batch.pos[batch.edge_index[0]] - batch.pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
        dist_TS = (batch.y_pos[batch.edge_index[0]] - batch.y_pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
        l_gate = F.binary_cross_entropy_with_logits(gate_logits, (torch.abs(dist_TS - dist_R) > 0.1).float())
        
        total_loss = 2.0 * diff_loss + 1.0 * l_gate + 0.05 * l_energy
        total_loss.backward()
        self.optimizer.step()
        
        return total_loss.item(), target_fwd, pred_fwd

Train

In [376]:
def collate_fn_filter_none(batch):
    batch = [data for data in batch if data is not None]
    if len(batch) == 0: return None
    return pyg_data.Batch.from_data_list(batch)

In [377]:
def train_fidelity_transfer_with_es(rgd1_path, t1x_path, rgd1_config, t1x_config, 
                                   pre_epochs=50, ft_epochs=100, limit=None, patience=20):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BEPGeoModel(node_dim=27, hidden_dim=256, num_layers=4).to(device)
    
    # 반환 변수 초기화 (에러 방지)
    hist = {'loss': []}
    r_E3 = {'E': []}
    p_E3 = {'E': []}

    # --- 파일 존재 여부 선제 확인 ---
    if not os.path.exists(rgd1_path):
        print(f"❌ 오류: RGD1 파일을 찾을 수 없습니다: {rgd1_path}")
        return hist, r_E3, p_E3
    if not os.path.exists(t1x_path):
        print(f"❌ 오류: T1x 파일을 찾을 수 없습니다: {t1x_path}")
        return hist, r_E3, p_E3

    # --- Phase 1: Pre-training (RGD1) ---
    print("\n" + "="*60 + "\n🚀 Phase 1: Pre-training on RGD1\n" + "="*60)
    rgd1_ds = ReactionDataset(rgd1_path, config=rgd1_config, limit=limit)
    rgd1_loader = DataLoader(rgd1_ds, batch_size=32, shuffle=True, collate_fn=collate_fn_filter_none)
    
    if len(rgd1_ds) == 0:
        print("⚠️ RGD1 데이터셋이 비어 있습니다.")
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        trainer = BEPTrainer(model, optimizer, device, pretrain_mode=False)
        for epoch in range(pre_epochs):
            model.train()
            count = 0
            pbar = tqdm(rgd1_loader, desc=f"[Pre] Epoch {epoch+1}/{pre_epochs}")
            for batch in pbar:
                if batch is None: continue
                loss, _, _ = trainer.train_step(batch)
                count += 1
            if count == 0: 
                print(f"⚠️ Epoch {epoch+1}: 유효한 배치가 없습니다.")
                break
        torch.save(model.state_dict(), "best_rgd1_pretrained.pth")

    # --- Phase 2: Fine-tuning (T1x) ---
    print("\n" + "="*60 + "\n🎯 Phase 2: Fine-tuning on T1x\n" + "="*60)
    t1x_ds = ReactionDataset(t1x_path, config=t1x_config, limit=limit)
    t1x_loader = DataLoader(t1x_ds, batch_size=32, shuffle=True, collate_fn=collate_fn_filter_none)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    trainer = BEPTrainer(model, optimizer, device, pretrain_mode=False)
    
    best_loss = float('inf')
    early_stop_count = 0

    for epoch in range(ft_epochs):
        model.train()
        train_loss, count = 0, 0
        pbar = tqdm(t1x_loader, desc=f"[Fine] Epoch {epoch+1}/{ft_epochs}")
        for batch in pbar:
            if batch is None: continue
            loss, y_true, y_pred = trainer.train_step(batch)
            train_loss += loss
            count += 1
            
            # 마지막 에폭 혹은 매 에폭 결과를 리스트에 수집 (시각화용)
            if epoch == ft_epochs - 1 or early_stop_count >= patience - 1:
                r = y_true.detach().cpu().numpy()
                p = y_pred.detach().cpu().numpy()
                r_E3['E'].extend(r[:, 2] if r.ndim > 1 else r.flatten())
                p_E3['E'].extend(p[:, 2] if p.ndim > 1 else p.flatten())

        if count > 0:
            avg_loss = train_loss / count
            hist['loss'].append(avg_loss)
            if avg_loss < best_loss:
                best_loss = avg_loss
                early_stop_count = 0
            else:
                early_stop_count += 1
            
            if early_stop_count >= patience:
                print(f"🛑 Early stopping at epoch {epoch+1}")
                break
        else:
            print("⚠️ T1x 유효 배치 없음.")
            break

    return hist, r_E3, p_E3

Early stopping

In [378]:
class EarlyStopping:
    """
    Validation Loss가 더 이상 개선되지 않으면 학습을 조기 종료시키는 클래스
    """
    def __init__(self, patience=20, verbose=False, delta=0, path='checkpoint.pth'):
        """
        Args:
            patience (int): Loss가 개선되지 않아도 참을 Epoch 수
            verbose (bool): True면 저장 메시지 출력
            delta (float): 개선으로 인정되는 최소 변화량
            path (str): 모델 저장 경로
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

Loop

In [ ]:
if __name__ == "__main__":
    # ... (기존 경로 설정 코드 유지) ...
    
    if os.path.exists(RGD1_PATH) and os.path.exists(T1X_PATH):
        hist, r_E3, p_E3 = train_fidelity_transfer_with_es(
            RGD1_PATH, T1X_PATH, RGD1_CONFIG, T1X_CONFIG, 
            pre_epochs=500,   # 넉넉하게 설정
            ft_epochs=500,    # 넉넉하게 설정
            limit=10000,       # 가능하면 None으로 전체 데이터 사용
            patience=20       # Early Stopping 감도
        )
        
        if hist:
            print_evaluation_report(r_E3, p_E3)


🚀 Phase 1: Pre-training on RGD1


[Pre] Epoch 136/500: 100%|███████████████████████████████████████████████████████████| 313/313 [03:20<00:00,  1.56it/s]


[Pre] Epoch 196/500:  23%|█████████████▊                                              | 72/313 [00:46<02:42,  1.49it/s]

시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def print_evaluation_report(real_dict, pred_dict):
    """
    정답(real_dict)과 예측값(pred_dict)을 비교하여 
    MAE, RMSE, R2 Score를 계산하고 시각화(Parity Plot)를 수행합니다.
    """
    # 1. Metric 계산 및 표 출력
    print("\n" + "="*85)
    print(f"   📊 FINAL PHYSICS-INFORMED MODEL PERFORMANCE REPORT (kcal/mol)")
    print("="*85)
    print(f"{'Energy Type':<15} | {'MAE':<10} | {'RMSE':<10} | {'R² Score':<10}")
    print("-" * 85)

    keys_to_check = ['H', 'G', 'E']
    active_keys = []

    for key in keys_to_check:
        if key not in real_dict or len(real_dict[key]) == 0:
            continue
        
        # 데이터 정제: 텐서일 경우 CPU로 이동 및 넘파이 변환
        try:
            y_true = np.array([v.cpu().detach().item() if torch.is_tensor(v) else v for v in real_dict[key]]).flatten()
            y_pred = np.array([v.cpu().detach().item() if torch.is_tensor(v) else v for v in pred_dict[key]]).flatten()
        except:
            y_true = np.array(real_dict[key]).flatten()
            y_pred = np.array(pred_dict[key]).flatten()

        # NaN 및 무한대 값 제거 (안전장치)
        mask = np.isfinite(y_true) & np.isfinite(y_pred)
        y_true, y_pred = y_true[mask], y_pred[mask]

        if len(y_true) == 0:
            continue
            
        # 메트릭 계산
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        
        print(f"Delta {key:<9} | {mae:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")
        active_keys.append((key, y_true, y_pred))
    
    print("="*85)

    # 2. 시각화 (Parity Plots)
    if not active_keys:
        print("⚠️ 출력할 유효한 데이터가 없습니다. 학습이 정상적으로 완료되었는지 확인하세요.")
        return

    num_plots = len(active_keys)
    fig, axes = plt.subplots(1, num_plots, figsize=(6 * num_plots, 6))
    
    if num_plots == 1:
        axes = [axes]

    for i, (key, y_true, y_pred) in enumerate(active_keys):
        # 산점도
        axes[i].scatter(y_true, y_pred, alpha=0.4, s=20, color='#3498db', edgecolors='white', linewidth=0.5)
        
        # 1:1 Reference Line (정답선)
        all_vals = np.concatenate([y_true, y_pred])
        mn, mx = all_vals.min(), all_vals.max()
        padding = (mx - mn) * 0.05
        limits = [mn - padding, mx + padding]
        
        axes[i].plot(limits, limits, color='#e74c3c', linestyle='--', lw=2, label='Ideal (1:1)')
        
        # 그래프 서식
        axes[i].set_xlim(limits)
        axes[i].set_ylim(limits)
        axes[i].set_title(f"Parity Plot: Delta {key}", fontsize=15, fontweight='bold')
        axes[i].set_xlabel("DFT Calculated (kcal/mol)", fontsize=12)
        axes[i].set_ylabel("Model Predicted (kcal/mol)", fontsize=12)
        axes[i].grid(True, linestyle=':', alpha=0.7)
        axes[i].legend()

    plt.tight_layout()
    plt.show()

# [실행 예시]
print_evaluation_report(r_E3, p_E3)